In [65]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import hvplot.pandas

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

In [2]:
top_5_mentions_crypto = requests.get('https://apewisdom.io/api/v1.0/filter/cryptocurrency').json()['results'][:5]

In [3]:
top_5_mentions_crypto

[{'rank': 1,
  'ticker': 'BTC.X',
  'name': 'Bitcoin',
  'mentions': '630',
  'upvotes': '1762',
  'rank_24h_ago': '1',
  'mentions_24h_ago': '921'},
 {'rank': 2,
  'ticker': 'UST.X',
  'name': 'TerraUSD',
  'mentions': '360',
  'upvotes': '4296',
  'rank_24h_ago': '3',
  'mentions_24h_ago': '369'},
 {'rank': 3,
  'ticker': 'ETH.X',
  'name': 'Ethereum',
  'mentions': '337',
  'upvotes': '1238',
  'rank_24h_ago': '2',
  'mentions_24h_ago': '511'},
 {'rank': 4,
  'ticker': 'LUNA.X',
  'name': 'Terra',
  'mentions': '314',
  'upvotes': '3917',
  'rank_24h_ago': '4',
  'mentions_24h_ago': '320'},
 {'rank': 5,
  'ticker': 'USDT.X',
  'name': 'Tether',
  'mentions': '125',
  'upvotes': '595',
  'rank_24h_ago': '5',
  'mentions_24h_ago': '246'}]

In [4]:
top_5_mentions_crypto_df = pd.DataFrame.from_records(top_5_mentions_crypto)
top_5_mentions_crypto_df

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,BTC.X,Bitcoin,630,1762,1,921
1,2,UST.X,TerraUSD,360,4296,3,369
2,3,ETH.X,Ethereum,337,1238,2,511
3,4,LUNA.X,Terra,314,3917,4,320
4,5,USDT.X,Tether,125,595,5,246


In [5]:
type(top_5_mentions_crypto_df['ticker'].iloc[0])

str

In [6]:
top_5_mentions_crypto_df['ticker'] = top_5_mentions_crypto_df['ticker'].str[:-2]


In [7]:
top_tickers = top_5_mentions_crypto_df.loc[:,'ticker']

In [8]:
# https://www.adamsmith.haus/python/answers/how-to-convert-a-column-in-a-pandas-dataframe-to-a-numpy-array-in-python#:~:text=Use%20pandas.,Use%20pandas.
# create variable to call for values only ** was getting error message because, for some reason-although this error was not present in the social meme file, it was not allowing to put data in a list using a for loop.

In [9]:
top_tickers_crypto = top_tickers.values
top_tickers_crypto

array(['BTC', 'UST', 'ETH', 'LUNA', 'USDT'], dtype=object)

In [10]:
top_5_mentions_crypto_df.set_index('ticker', inplace=True)
top_5_mentions_crypto_df

,rank,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,
BTC,1,Bitcoin,630,1762,1,921
UST,2,TerraUSD,360,4296,3,369
ETH,3,Ethereum,337,1238,2,511
LUNA,4,Terra,314,3917,4,320
USDT,5,Tether,125,595,5,246


In [11]:
top_5_mentions_crypto_df = pd.DataFrame.from_dict(top_5_mentions_crypto_df, orient='columns')
top_5_mentions_crypto_df

,rank,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,
BTC,1,Bitcoin,630,1762,1,921
UST,2,TerraUSD,360,4296,3,369
ETH,3,Ethereum,337,1238,2,511
LUNA,4,Terra,314,3917,4,320
USDT,5,Tether,125,595,5,246


In [12]:
top_5_mentions_crypto_df.drop(columns=['rank','name','upvotes','rank_24h_ago','mentions_24h_ago'],inplace=True)                              
top_5_mentions_crypto_df

,mentions
ticker,
BTC,630
UST,360
ETH,337
LUNA,314
USDT,125


In [13]:
type(top_5_mentions_crypto_df['mentions'].iloc[0])

str

In [14]:
top_5_mentions_crypto_df = pd.to_numeric(top_5_mentions_crypto_df['mentions'], errors='coerce') #ya!!

In [28]:
top_5_mentions_crypto_df.index[0]

'BTC'

In [15]:
top_5_mentions_crypto_df.hvplot(kind="bar", title='Mentions in last 24 hours for top 5 cryptocurrencies')

:Bars   [ticker]   (mentions)

In [78]:
top_crypto_tickers = []

for crypto in top_5_mentions_crypto_df.index:
    top_crypto_tickers.append(crypto+'-USD')
top_crypto_tickers

tickers_for_yf = ''
for crypto in top_crypto_tickers:
    tickers_for_yf = tickers_for_yf + f' {crypto}'
tickers_for_yf

' BTC-USD UST-USD ETH-USD LUNA-USD USDT-USD'

In [17]:
load_dotenv('crypto.env')

True

In [18]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
type(alpaca_api_key)

str

In [ ]:
# https://alpaca.markets/docs/broker/integration/crypto-trading-broker/ # alpaca information about tickers and exchanges

In [ ]:
# https://github.com/alpacahq/alpaca-trade-api-python/blob/master/alpaca_trade_api/rest.py #INFORMATION ABOUT HOE GET_CRYPTO_BARS WORKS ... AND OTHERS...

In [ ]:
url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'b54bcf4d-1bca-4e8e-9a24-22ff2c3d462c',
}

In [59]:
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = 'v2')

start_date = pd.Timestamp('2022-05-14', tz='America/New_York').isoformat()
# end_date = pd.Timestamp('2022-05-12', tz='America/New_York').isoformat() 
prices_df = alpaca.get_crypto_bars(top_crypto_tickers, timeframe = '15Min', start = start_date, exchanges = ['CBSE']).df

BTC = prices_df[prices_df['symbol'] == 'BTCUSD'].drop('symbol', axis = 1)
LUNA = prices_df[prices_df['symbol'] == 'LUNAUSD'].drop('symbol', axis = 1)
ETH = prices_df[prices_df['symbol'] == 'ETHUSD'].drop('symbol', axis = 1)
UST = prices_df[prices_df['symbol'] == 'USTUSD'].drop('symbol', axis = 1)
USDT = prices_df[prices_df['symbol'] == 'USDTUSD'].drop('symbol', axis = 1)

# BTC = prices_df[prices_df['symbol'] == 'BTC']
# LUNA = prices_df[prices_df['symbol'] == 'LUNA']
# ETH = prices_df[prices_df['symbol'] == 'ETH']
# UST = prices_df[prices_df['symbol'] == 'UST']
# USDT = prices_df[prices_df['symbol'] == 'USDT']


prices_df = pd.concat([BTC['close'], LUNA['close'], ETH['close'], UST['close'], USDT['close']], axis = 1, keys = ['BTC', 'LUNA', 'ETH', 'UST', 'ADA'])
prices_df.head()

,BTC,LUNA,ETH,UST,ADA
timestamp,,,,,
2022-05-14 04:00:00+00:00,29494.98,NaN,2044.91,NaN,NaN
2022-05-14 04:15:00+00:00,29448.83,NaN,2040.03,NaN,NaN
2022-05-14 04:30:00+00:00,29307.01,NaN,2025.98,NaN,NaN
2022-05-14 04:45:00+00:00,29294.90,NaN,2018.85,NaN,NaN
2022-05-14 05:00:00+00:00,29340.05,NaN,2025.63,NaN,NaN


In [6]:
# prices_df_plot = (prices_df.pct_change()).hvplot()
prices_df_plot = (1 + prices_df.pct_change()).cumprod().hvplot()
prices_df_plot

NameError: name 'prices_df' is not defined

In [7]:
top_5_crypto_prices = {'mentions': int(top_5_mentions_crypto[0]['mentions']), 'mentions_24h_ago': int(top_5_mentions_crypto[0]['mentions_24h_ago'])}
top_5_crypto_prices

NameError: name 'top_5_mentions_crypto' is not defined

In [3]:
mentions_top_5_crypto_plot = top_5_mentions_crypto_df.hvplot(kind='bar')
mentions_top_5_crypto_plot

NameError: name 'top_5_mentions_crypto_df' is not defined

In [4]:
overlayplot = prices_df_plot + mentions_top_5_crypto_plot


NameError: name 'prices_df_plot' is not defined

In [5]:
overlayplot

NameError: name 'overlayplot' is not defined

In [56]:
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = 'v2')

start_date = pd.Timestamp('2022-05-14', tz='America/New_York').isoformat()
# end_date = pd.Timestamp('2022-05-12', tz='America/New_York').isoformat() 
prices_df = alpaca.get_crypto_bars('USTUSD', timeframe = '1Hour', start = start_date, exchanges = ['CBSE']).df
prices_df

""


In [57]:
prices_df = alpaca.get_crypto_bars(top_crypto_tickers, timeframe = '15Min', start = start_date).df

In [58]:
prices_df

,exchange,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,,
2022-05-14 04:00:00+00:00,CBSE,29513.4700,29527.3100,29437.8700,29494.9800,87.602716,4747,29469.955509,BTCUSD
2022-05-14 04:00:00+00:00,ERSX,29465.4000,29503.5000,29464.5000,29495.8000,0.200000,5,29470.343605,BTCUSD
2022-05-14 04:00:00+00:00,FTXU,29526.0000,29529.0000,29460.0000,29529.0000,1.015400,20,29484.355131,BTCUSD
2022-05-14 04:15:00+00:00,CBSE,29493.3900,29528.4000,29438.2500,29448.8300,88.485190,4691,29483.528444,BTCUSD
2022-05-14 04:15:00+00:00,ERSX,29518.1000,29520.9000,29482.5000,29490.8000,0.085700,11,29506.468145,BTCUSD
...,...,...,...,...,...,...,...,...,...
2022-05-17 15:00:00+00:00,FTXU,0.9989,0.9989,0.9989,0.9989,14972.580000,2,0.998900,USDTUSD
2022-05-17 15:30:00+00:00,FTXU,0.9988,0.9989,0.9988,0.9989,5.650000,2,0.998800,USDTUSD
2022-05-17 15:45:00+00:00,FTXU,0.9988,0.9989,0.9988,0.9989,3483.500000,10,0.998863,USDTUSD


In [74]:
import yfinance as yf
msft = yf.Ticker('BTC-USD')


In [81]:
data = yf.download(tickers_for_yf, start = '2022-05-16', interval = '15m')


[*********************100%***********************]  5 of 5 completed

1 Failed download:
- LUNA-USD: No data found for this date range, symbol may be delisted


,BTC-USD,ETH-USD,LUNA-USD,USDT-USD,UST-USD
2022-05-16 06:00:00+00:00,29966.171875,2046.325684,NaN,0.998819,0.171980
2022-05-16 06:15:00+00:00,29541.869141,2020.854126,NaN,0.998782,0.169351
2022-05-16 06:30:00+00:00,29524.328125,2019.137573,NaN,0.998848,0.167053
2022-05-16 06:45:00+00:00,29522.140625,2011.563965,NaN,0.998973,0.166404
2022-05-16 07:00:00+00:00,29443.396484,2002.653076,NaN,0.998931,0.165256
...,...,...,...,...,...
2022-05-17 16:15:00+00:00,30012.810547,2046.253662,NaN,0.998781,0.116941
2022-05-17 16:30:00+00:00,30074.013672,2052.571045,NaN,0.998788,0.117274
2022-05-17 16:45:00+00:00,30156.507812,2058.908447,NaN,0.998828,0.117566
2022-05-17 16:56:00+00:00,NaN,2058.775391,NaN,0.998819,0.117541


In [82]:
data['Close']

,BTC-USD,ETH-USD,LUNA-USD,USDT-USD,UST-USD
2022-05-16 06:00:00+00:00,29966.171875,2046.325684,NaN,0.998819,0.171980
2022-05-16 06:15:00+00:00,29541.869141,2020.854126,NaN,0.998782,0.169351
2022-05-16 06:30:00+00:00,29524.328125,2019.137573,NaN,0.998848,0.167053
2022-05-16 06:45:00+00:00,29522.140625,2011.563965,NaN,0.998973,0.166404
2022-05-16 07:00:00+00:00,29443.396484,2002.653076,NaN,0.998931,0.165256
...,...,...,...,...,...
2022-05-17 16:15:00+00:00,30012.810547,2046.253662,NaN,0.998781,0.116941
2022-05-17 16:30:00+00:00,30074.013672,2052.571045,NaN,0.998788,0.117274
2022-05-17 16:45:00+00:00,30156.507812,2058.908447,NaN,0.998828,0.117566
2022-05-17 16:56:00+00:00,NaN,2058.775391,NaN,0.998819,0.117541


In [1]:
graph = (1 + data['Close'].pct_change()).cumprod().hvplot()
graph

NameError: name 'data' is not defined

In [2]:

import requests
url = "https://rest.yahoofinanceapi.com/v6/finance/quote"
querystring = {"symbols":"BTC-USD,EURUSD=X"}
headers = {
'x-api-key': "YOUR-PERSONAL-API-KEY"
}
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.text)

<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="/favicon.ico"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="theme-color" content="#000000"/><meta name="description" content="Alternatives to Yahoo Finance API with documentation and code samples"/><link rel="apple-touch-icon" href="/logo192.png"/><link rel="manifest" href="/manifest.json"/><title>Yahoo Finance API alternatives</title><script async src="https://www.googletagmanager.com/gtag/js?id=UA-199855457-1"></script><script>function gtag(){dataLayer.push(arguments)}window.dataLayer=window.dataLayer||[],gtag("js",new Date),gtag("config","UA-199855457-1")</script><link href="/static/css/2.d082992b.chunk.css" rel="stylesheet"><link href="/static/css/main.77d41950.chunk.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(e){function r(r){for(var n,a,i=r[0],l=r[1],p=r[2],c=0,s=[];c<